# PREDICTION OF KROGER'S STOCK (KR)

## Uploading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_raw = pd.read_csv("/Users/phuonglucydoan/Desktop/STUDY/Spring 2020/Courses/IMSE 8370 _ Supply Chain Modeling and Analysis/Individual Project/Stock Price Prediction/KR.csv"
                  , index_col="Date")

In [3]:
data_raw.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5120.000000,5120.000000,5120.000000,5120.000000,5120.000000,5.120000e+03
mean,16.969381,17.163529,16.781272,16.976040,14.436098,9.775719e+06
std,9.209952,9.298238,9.122698,9.211398,9.607939,6.529938e+06
min,6.100000,6.205000,5.500000,5.705000,3.928635,1.152000e+06
25%,10.375000,10.498750,10.250000,10.375000,7.574934,5.815750e+06
50%,12.307500,12.490000,12.175000,12.325000,9.335241,8.195000e+06
75%,24.035001,24.287501,23.793750,24.010000,22.649848,1.185975e+07
max,42.689999,42.750000,42.330002,42.639999,39.384224,1.541652e+08


In [4]:
data_DJIA = pd.read_csv("/Users/phuonglucydoan/Desktop/STUDY/Spring 2020/Courses/IMSE 8370 _ Supply Chain Modeling and Analysis/Individual Project/Stock Price Prediction/DJI.csv"
                  , index_col="Date")

In [5]:
data_raw['DJIA_Open']=data_DJIA['Open']

In [6]:
data_raw.head(5)

,Open,High,Low,Close,Adj Close,Volume,DJIA_Open
Date,,,,,,,
2000-01-03,9.56250,9.62500,9.37500,9.46875,6.520464,11339200,11501.849609
2000-01-04,9.40625,9.84375,9.37500,9.78125,6.735664,17990600,11349.750000
2000-01-05,9.71875,10.09375,9.68750,9.93750,6.843263,15291000,10989.370117
2000-01-06,9.93750,9.93750,9.28125,9.56250,6.585027,17686400,11113.370117
2000-01-07,9.50000,9.62500,9.34375,9.46875,6.520464,13358200,11247.059570


## Generating features

In [7]:
#Create sub-function that generates 6 original features

def add_original_feature (df, df_new):
    df_new["open"] = df["Open"]
    df_new["open_1"] = df["Open"].shift(1)
    df_new["close_1"] = df["Close"].shift(1)
    df_new["high_1"] = df["High"].shift(1)
    df_new["low_1"] = df["Low"].shift(1)
    df_new["volume_1"] = df["Volume"].shift(1)

In [8]:
#Create sub-function that generates features related to average close prices

def add_avg_price (df, df_new):
    df_new["avg_price_5"] = df["Close"].rolling(5).mean().shift(1)
    df_new["avg_price_30"] = df["Close"].rolling(21).mean().shift(1)
    df_new["avg_price_365"] = df["Close"].rolling(252).mean().shift(1)
    df_new["ratio_avg_price_5_30"] = df_new["avg_price_5"] / df_new["avg_price_30"]
    df_new["ratio_avg_price_5_365"] = df_new["avg_price_5"] / df_new["avg_price_365"]
    df_new["ratio_avg_price_30_365"] = df_new["avg_price_30"] / df_new["avg_price_365"]

In [9]:
#Create sub-function that generates features related to average volumes

def add_avg_volume (df, df_new):
    df_new["avg_volume_5"] = df["Volume"].rolling(5).mean().shift(1)
    df_new["avg_volume_30"] = df["Volume"].rolling(21).mean().shift(1)
    df_new["avg_volume_365"] = df["Volume"].rolling(252).mean().shift(1)
    df_new["ratio_avg_volume_5_30"] = df_new["avg_volume_5"] / df_new["avg_volume_30"]
    df_new["ratio_avg_volume_5_365"] = df_new["avg_volume_5"] / df_new["avg_volume_365"]
    df_new["ratio_avg_volume_30_365"] = df_new["avg_volume_30"] / df_new["avg_volume_365"]

In [10]:
#Create sub-function that generates features related to standard deviation of close prices

def add_std_price (df, df_new):
    df_new["std_price_5"] = df["Close"].rolling(5).std().shift(1)
    df_new["std_price_30"] = df["Close"].rolling(21).std().shift(1)
    df_new["std_price_365"] = df["Close"].rolling(252).std().shift(1)
    df_new["ratio_stdg_price_5_30"] = df_new["std_price_5"] / df_new["std_price_30"]
    df_new["ratio_std_price_5_365"] = df_new["std_price_5"] / df_new["std_price_365"]
    df_new["ratio_std_price_30_365"] = df_new["std_price_30"] / df_new["std_price_365"]

In [11]:
#Create sub-function that generates features related to standard deviation of volumes

def add_std_volume (df, df_new):
    df_new["std_volume_5"] = df["Volume"].rolling(5).std().shift(1)
    df_new["std_volume_30"] = df["Volume"].rolling(21).std().shift(1)
    df_new["std_volume_365"] = df["Volume"].rolling(252).std().shift(1)
    df_new["ratio_stdg_volume_5_30"] = df_new["std_volume_5"] / df_new["std_volume_30"]
    df_new["ratio_std_volume_5_365"] = df_new["std_volume_5"] / df_new["std_volume_365"]
    df_new["ratio_std_volume_30_365"] = df_new["std_volume_30"] / df_new["std_volume_365"]

In [12]:
weights = np.arange(1,6)
weights

array([1, 2, 3, 4, 5])

In [13]:
#Create sub-function that generates 9 return-based features

def add_return_feature (df, df_new):
    df_new["return_1"] = ((df["Close"] - df["Close"].shift(1))/df["Close"].shift(1)).shift(1)
    df_new["return_5"] = ((df["Close"] - df["Close"].shift(5))/df["Close"].shift(5)).shift(1)
    df_new["return_30"] = ((df["Close"] - df["Close"].shift(21))/df["Close"].shift(21)).shift(1)
    df_new["return_365"] = ((df["Close"] - df["Close"].shift(252))/df["Close"].shift(252)).shift(1)
    df_new["moving_avg_5"] = df_new["return_1"].rolling(5).mean().shift(1)
    df_new["moving_avg_30"] = df_new["return_1"].rolling(21).mean().shift(1)
    df_new["moving_avg_365"] = df_new["return_1"].rolling(255).mean().shift(1)
    df_new["weighted_moving_avg_5"] = df_new["return_1"].rolling(5).apply(lambda wma_return: np.dot(wma_return, weights)/weights.sum(), raw=True)
    df_new["exp_smoothing_5"] = df_new["moving_avg_5"].ewm(span=5).mean().shift(1)

In [14]:
#The main feature generating function that combines all sub-functions

def generate_feature(df):
    df_new=pd.DataFrame()
    add_original_feature(df, df_new)
    add_avg_price(df, df_new)
    add_avg_volume(df, df_new)
    add_std_price(df, df_new)
    add_std_volume(df, df_new)
    add_return_feature(df, df_new)
    
    df_new = df_new.dropna(axis=0)

    df_new["close"] = df["Close"]
    df_new["DJIA_open"] = df["DJIA_Open"]
    return df_new

In [15]:
data = generate_feature(data_raw)

In [16]:
data.describe()

,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,weighted_moving_avg_5,exp_smoothing_5,close,DJIA_open
count,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4.863000e+03,4863.000000,4863.000000,4863.000000,4863.000000,...,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000,4863.000000
mean,17.322507,17.318187,17.323765,17.510810,17.130315,9.910181e+06,17.315209,17.283479,16.879815,1.001674,...,0.001710,0.006604,0.081356,0.000340,0.000340,0.000347,0.000354,0.000334,17.328153,14424.742829
std,9.311083,9.308820,9.310822,9.401511,9.217796,6.619627e+06,9.301129,9.267542,8.962674,0.032627,...,0.037666,0.071347,0.262599,0.007572,0.003408,0.000916,0.008375,0.005673,9.313341,5463.893193
min,6.100000,6.100000,5.705000,6.205000,5.500000,1.358400e+06,6.221000,6.612143,7.673770,0.804780,...,-0.275828,-0.378202,-0.538804,-0.059472,-0.021603,-0.002761,-0.081784,-0.046462,5.705000,6547.009766
25%,10.557500,10.557500,10.570000,10.692500,10.427500,5.888900e+06,10.590500,10.614048,10.794345,0.985422,...,-0.016751,-0.034709,-0.113458,-0.003264,-0.001564,-0.000309,-0.003559,-0.002384,10.570000,10414.839844
50%,12.500000,12.500000,12.515000,12.675000,12.380000,8.303800e+06,12.519000,12.481429,12.135947,1.004176,...,0.002352,0.011526,0.044619,0.000545,0.000660,0.000309,0.000696,0.000632,12.520000,12417.959961
75%,24.562500,24.559999,24.582499,24.784999,24.340000,1.202780e+07,24.564500,24.645238,25.334405,1.021323,...,0.022186,0.052814,0.252605,0.004462,0.002560,0.001035,0.004883,0.003565,24.587499,17590.355469
max,42.689999,42.689999,42.639999,42.750000,42.330002,1.541652e+08,42.434000,41.813810,37.725556,1.116569,...,0.306748,0.312883,1.018336,0.055420,0.013533,0.002828,0.058342,0.030012,42.639999,29440.470703


In [17]:
data.round(decimals=3).head(5)

,open,open_1,close_1,high_1,low_1,volume_1,avg_price_5,avg_price_30,avg_price_365,ratio_avg_price_5_30,...,return_5,return_30,return_365,moving_avg_5,moving_avg_30,moving_avg_365,weighted_moving_avg_5,exp_smoothing_5,close,DJIA_open
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-09,12.000,11.688,12.000,12.219,11.656,6178400.0,12.175,12.766,10.327,0.954,...,-0.113,0.003,0.267,-0.028,-0.002,0.001,-0.011,-0.008,11.938,10625.21
2001-01-10,11.906,12.000,11.938,12.188,11.906,6271600.0,11.894,12.734,10.337,0.934,...,-0.105,-0.054,0.295,-0.023,0.001,0.001,-0.005,-0.015,11.594,10568.48
2001-01-11,11.719,11.906,11.594,12.000,11.469,9000000.0,11.738,12.699,10.347,0.924,...,-0.063,-0.058,0.266,-0.021,-0.002,0.001,-0.008,-0.018,11.625,10600.20
2001-01-12,11.625,11.719,11.625,11.875,11.562,9040400.0,11.750,12.682,10.356,0.927,...,0.005,-0.031,0.253,-0.012,-0.002,0.001,-0.003,-0.019,11.219,10608.74
2001-01-16,11.156,11.625,11.219,11.625,11.094,7898800.0,11.675,12.621,10.363,0.925,...,-0.032,-0.102,0.181,0.001,-0.001,0.001,-0.015,-0.017,11.719,10525.78


## Data Splitting and Pre-processing

In [18]:
#Splitting data and counting training samples

start_train = '2001-01-09'
end_train = '2018-12-31'
start_test = '2019-01-02'
end_test = '2020-05-08'
data_train = data.loc[start_train:end_train]
X_train = data_train.drop('close', axis=1).values
y_train = data_train['close'].values

print(X_train.shape, y_train.shape)

(4522, 40) (4522,)


In [19]:
#Counting testing samples

data_test = data.loc[start_test:end_test]
X_test = data_test.drop('close', axis=1).values
y_test = data_test['close'].values

print(X_test.shape)

(341, 40)


In [20]:
#Rescaling both training and testing datasets

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

## Learning from Datasets and Making Predictions

In [21]:
#Install predicting algorithms and performance-evaluating metrics

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### SGD-based Linear Regression

In [23]:
#Find out optimal set of parameters for SGD-based Linear Regression model

param_grid = {
    "alpha": [1e-5, 3e-5, 1e-4],
    "eta0": [0.01, 0.03, 0.1],
}
lr = SGDRegressor(loss='squared_loss', learning_rate='constant', penalty='l2', max_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='r2')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)

{'alpha': 1e-05, 'eta0': 0.01}


In [24]:
#Use the optimal model to make predictions of the testing samples

lr_best = grid_search.best_estimator_
predictions_lr = lr_best.predict(X_scaled_test)

In [25]:
#Measure predicting performance of Linear Regression model

MSE_lr = mean_squared_error(y_test, predictions_lr).round(decimals=3)
MAE_lr = mean_absolute_error(y_test, predictions_lr).round(decimals=3)
Rsquared_lr = r2_score(y_test, predictions_lr).round(decimals=3)
print("For Linear Regression model,",
      "Mean squared error:", MSE_lr, ", Mean absolute error:", MAE_lr, ", R_squared:", Rsquared_lr)

For Linear Regression model, Mean squared error: 0.36 , Mean absolute error: 0.415 , R_squared: 0.962


### Regression Forest

In [ ]:
#Find out optimal set of parameters for Regression Forest model

param_grid = {
    'max_depth': [50,70,80],
    'min_samples_split': [5,10],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3,5]
}
rf = RandomForestRegressor(n_estimators=500, n_jobs=-1)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

In [26]:
#Use the optimal model to make predictions of the testing samples

rf_best = grid_search.best_estimator_
predictions_rf = rf_best.predict(X_test)

In [27]:
#Measure predicting performance of Regression Forest model

MSE_rf = mean_squared_error(y_test, predictions_rf).round(decimals=3)
MAE_rf = mean_absolute_error(y_test, predictions_rf).round(decimals=3)
Rsquared_rf = r2_score(y_test, predictions_rf).round(decimals=3)
print("For Regression Forest model,",
      "Mean squared error:", MSE_rf, ", Mean absolute error:", MAE_rf, ", R_squared:", Rsquared_rf)

For Regression Forest model, Mean squared error: 0.309 , Mean absolute error: 0.373 , R_squared: 0.967
